Loads the file names into a dict and pickles...

http://nptdms.readthedocs.io/en/latest/

In [8]:
data_dir = ".\\data\\"

#filename = "2018-06-20-10-44-11.tdms"
#keyence_filename= "2018-06-20-10-44-18-KEYENCE.tdms"

#filename = "2018-07-12-18-36-41.tdms"
#keyence_filename= "2018-07-12-18-36-56-KEYENCE.tdms"

filename = "2018-07-12-18-51-08.tdms"
keyence_filename= "2018-07-12-18-51-17-KEYENCE.tdms"

In [9]:
#Import SmarAct/Temperature data
from nptdms import TdmsFile

# Load the file
tdms_file = TdmsFile(data_dir + filename)

props = tdms_file.object().properties
props['Acceleration'] = props['Acceleration']/1e9
props['Velocity'] = props['Velocity']/1e9
props['SetPoint1'] = props['SetPoint1']/1e9
props['SetPoint2'] = props['SetPoint2']/1e9

info = \
"""Filename: \t{name}
Dwell time: \t{Dwell}s
Velocity: \t{Velocity}mm/s
Acceleration: \t{Acceleration}mm/s²
Max freq: \t{Freq}Hz
Setpoints: \t{SetPoint1}mm, {SetPoint2}mm""".format(**props)

print(info)

info_key, info_val = zip(*[i.split('\t') for i in info.split('\n')])

# Read the setpoint data
set_point = dict(
    time  = tdms_file.object('SetPoint', 'Time').data,
    data  = tdms_file.object('SetPoint', 'SET_PT').data
    )

# Read the position data
position = dict(
    time  = tdms_file.object('Position', 'Time').data,
    data  = tdms_file.object('Position', 'SM-POS').data
    )

# Convert pm into mm
set_point['data'] = [d/1e9 for d in set_point['data']]
position['data']  = [d/1e9 for d in position['data']]

# Read the temperature data
temperature = dict(
    time  = tdms_file.object('Temperature', 'Time').data,
    stage = tdms_file.object('Temperature', 'T_STAGE').data,
    base  = tdms_file.object('Temperature', 'T_BASE').data,
    amb   = tdms_file.object('Temperature', 'T_AMB').data
    )

Filename: 	2018-07-12-18-51-08
Dwell time: 	0.5s
Velocity: 	0.0mm/s
Acceleration: 	10.0mm/s²
Max freq: 	20000Hz
Setpoints: 	-8.0mm, -12.0mm


In [10]:
# Import Keyence data
from nptdms import TdmsFile

# Load the file
keyence_tdms_file = TdmsFile(data_dir + keyence_filename)

keyence = dict(
    time  = keyence_tdms_file.object('DATA', 'Time').data,
    # The keyence data is -ve - invert it!
    data1 = keyence_tdms_file.object('DATA', 'OUT1').data * -1,
    data2 = keyence_tdms_file.object('DATA', 'OUT2').data * -1
)

In [11]:
full_data = dict(
    set_point   = set_point,
    position    = position,
    temperature = temperature,
    keyence     = keyence,
    info        = dict([i.split('\t') for i in info.split('\n')])
)

In [12]:
print("Full dataset:")
for k in full_data:
    print("{}: \t{} points".format(k, len(full_data[k][[*full_data[k]][0]])))

Full dataset:
set_point: 	1527 points
position: 	24196 points
temperature: 	1216 points
keyence: 	26738 points
info: 	19 points


In [13]:
for k in full_data:
    print(k)
    for l in full_data[k]:
        print("\t"+l)

set_point
	time
	data
position
	time
	data
temperature
	time
	stage
	base
	amb
keyence
	time
	data1
	data2
info
	Filename: 
	Dwell time: 
	Velocity: 
	Acceleration: 
	Max freq: 
	Setpoints: 


In [14]:
# Dump the data to file...

import pickle

pickle_file = data_dir + "_".join([filename, keyence_filename, "full_data.p"])

pickle.dump(full_data, open(pickle_file, 'wb'))

print("Full dataset stored in {}".format(pickle_file))

Full dataset stored in .\data\2018-07-12-18-51-08.tdms_2018-07-12-18-51-17-KEYENCE.tdms_full_data.p
